In [1]:
#импорт библиотек и первичный анализ датасета
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
import joblib

df = pd.read_csv('C:/Users/curse/Documents/aieprog/aie-repository/homeworks/HW06/S06-hw-dataset-02.csv')

print(df.head())
print(df.info())
print(df.describe(include='all'))

target_col = 'target'
print(df[target_col].value_counts(normalize=True))

print("Пропуски:\n", df.isnull().sum())
print("Типы столбцов:\n", df.dtypes)

   id       f01       f02       f03        f04       f05       f06       f07  \
0   1 -0.149235 -2.826966 -0.522901  -4.198449  1.364943  0.815043 -1.195518   
1   2 -1.966180 -4.877542  0.268367  -9.607791  0.097149  1.347185 -3.872575   
2   3 -0.555964 -0.999920  0.209673 -14.119498 -1.808950 -0.006222 -4.651108   
3   4 -2.049199 -5.600713 -1.664677  -6.263893 -5.224455  0.848351  1.407210   
4   5 -0.220556  4.889479 -2.235840   6.450046  0.774389 -2.382625  2.584816   

        f08       f09  ...       f29       f30       f31       f32       f33  \
0 -1.932232  2.396353  ... -0.159323  0.448015  0.572745  0.149916  0.878392   
1 -0.395117  1.710068  ... -0.389212  1.383794  0.169876  0.043969 -0.963545   
2  0.911944 -0.289037  ... -1.383970  3.044321 -0.182864  1.425649 -8.418598   
3 -0.542080  0.119102  ... -2.713080  2.762637 -0.520796 -0.142455  1.668338   
4  4.211856 -0.317889  ... -1.302872  2.478862  1.528610  1.098131  3.547087   

        f34        f35    x_int_1    x

Провел первичный анализ датасета, пропусков нет, таргет 73.7% на 26.3%, то есть не совсем сбалансированный.

In [2]:
# В этом блоке определяем X и y
X = df.drop(columns=['id', target_col])  # X - все столбцы кроме таргета и id 
y = df[target_col]

In [3]:
#Train/Test-сплит и воспроизводимость
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42,
    stratify=y
)

Мы фиксируем random_state (seed) чтобы при кажом запуске разбиение на test и train было одинаковым - это позволит нам сравнивать вывод честно и отлаживать код.
Стратификация сохраняет пропорции классов в train и test - в нашем случае это особенно важно так как в таргете наблюдается дисбаланс.

In [4]:
from sklearn.dummy import DummyClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, confusion_matrix

def print_metrics(y_true, y_pred, y_proba=None, title=None): #функция для вывода метрик из примерного ноутбука
    if title:
        print(title)
    acc = accuracy_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)
    print(f"accuracy = {acc:.4f}")
    print(f"f1       = {f1:.4f}")
    if y_proba is not None:
        try:
            auc = roc_auc_score(y_true, y_proba)
            print(f"roc_auc  = {auc:.4f}")
        except Exception:
            pass
    print("confusion_matrix:")
    print(confusion_matrix(y_true, y_pred))
    print()

# DummyClassifier
dummy = DummyClassifier(strategy='stratified', random_state=42)
dummy.fit(X_train, y_train)
y_pred_dummy = dummy.predict(X_test)
y_proba_dummy = dummy.predict_proba(X_test)[:, 1]
print_metrics(y_test, y_pred_dummy, y_proba=y_proba_dummy, title="Baseline: DummyClassifier (most_frequent)")

# LogisticRegression через Pipeline
lr_pipe = Pipeline([
    ('scaler', StandardScaler()),
    ('lr', LogisticRegression(random_state=42, max_iter=1000))
])
lr_pipe.fit(X_train, y_train)
y_pred_lr = lr_pipe.predict(X_test)
y_proba_lr = lr_pipe.predict_proba(X_test)[:, 1]
print_metrics(y_test, y_pred_lr, y_proba=y_proba_lr, title="Baseline: LogisticRegression")

Baseline: DummyClassifier (most_frequent)
accuracy = 0.6153
f1       = 0.2660
roc_auc  = 0.5027
confusion_matrix:
[[1964  691]
 [ 694  251]]

Baseline: LogisticRegression
accuracy = 0.8119
f1       = 0.5607
roc_auc  = 0.7977
confusion_matrix:
[[2491  164]
 [ 513  432]]



In [5]:
#Модели недели 6

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, HistGradientBoostingClassifier
from sklearn.model_selection import GridSearchCV

scoring = 'f1'

#DecisionTreeClassifier
dt = DecisionTreeClassifier(random_state=42)
dt_params = {
    'max_depth': [4, 6],          
    'min_samples_leaf': [5]       
}
dt_cv = GridSearchCV(dt, dt_params, cv=3, scoring=scoring, n_jobs=-1)
dt_cv.fit(X_train, y_train)
print_metrics(y_test, dt_cv.predict(X_test), dt_cv.predict_proba(X_test)[:, 1], title="DecisionTreeClassifier")

#RandomForestClassifier
rf = RandomForestClassifier(n_estimators=50, random_state=42)
rf_params = {
    'max_depth': [6, None],      
    'min_samples_leaf': [5]
}
rf_cv = GridSearchCV(rf, rf_params, cv=3, scoring=scoring, n_jobs=-1)
rf_cv.fit(X_train, y_train)
print_metrics(y_test, rf_cv.predict(X_test), rf_cv.predict_proba(X_test)[:, 1], title="RandomForestClassifier")

#HistGradientBoostingClassifier
hgb = HistGradientBoostingClassifier(random_state=42)
hgb_params = {
    'max_depth': [4],            
    'learning_rate': [0.1]        
}
hgb_cv = GridSearchCV(hgb, hgb_params, cv=3, scoring=scoring, n_jobs=-1)
hgb_cv.fit(X_train, y_train)
print_metrics(y_test, hgb_cv.predict(X_test), hgb_cv.predict_proba(X_test)[:, 1], title="HistGradientBoostingClassifier")

DecisionTreeClassifier
accuracy = 0.8106
f1       = 0.5727
roc_auc  = 0.8089
confusion_matrix:
[[2461  194]
 [ 488  457]]

RandomForestClassifier
accuracy = 0.8858
f1       = 0.7452
roc_auc  = 0.9201
confusion_matrix:
[[2588   67]
 [ 344  601]]

HistGradientBoostingClassifier
accuracy = 0.8872
f1       = 0.7578
roc_auc  = 0.9141
confusion_matrix:
[[2559   96]
 [ 310  635]]



In [6]:
#Вторая функция для отображения метрик качества в формате json

def evaluate_model(model, X_test, y_test, model_name):
    y_pred = model.predict(X_test)
    acc = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    
    if hasattr(model, "predict_proba"):
        y_proba = model.predict_proba(X_test)[:, 1]
        roc_auc = roc_auc_score(y_test, y_proba)
    else:
        roc_auc = None
    
    return {
        "model": model_name,
        "accuracy": float(acc),
        "f1": float(f1),
        "roc_auc": float(roc_auc) if roc_auc is not None else None
    }

metrics_list = [
    evaluate_model(dummy, X_test, y_test, "DummyClassifier"),
    evaluate_model(lr_pipe, X_test, y_test, "LogisticRegression"),
    evaluate_model(dt_cv, X_test, y_test, "DecisionTreeClassifier"),
    evaluate_model(rf_cv, X_test, y_test, "RandomForestClassifier"),
    evaluate_model(hgb_cv, X_test, y_test, "HistGradientBoostingClassifier")
]
metrics_dict = {m["model"]: {k: v for k, v in m.items() if k != "model"} for m in metrics_list}

with open('artifacts/metrics_test.json', 'w') as f: #Cохраняем общие метрики в отдельный файл
    json.dump(metrics_dict, f, indent=4)

In [7]:
#Графики и диаграммы

from sklearn.metrics import (
    roc_curve, precision_recall_curve, ConfusionMatrixDisplay
)

models = {
    "DummyClassifier": dummy,
    "LogisticRegression": lr_pipe,
    "DecisionTreeClassifier": dt_cv,
    "RandomForestClassifier": rf_cv,
    "HistGradientBoostingClassifier": hgb_cv
}

best_model_name = None
best_auc = -1
best_model = None

#Определяем лучшую модель по roc-auc
for name, model in models.items():
    y_proba = model.predict_proba(X_test)[:, 1]
    auc_score = roc_auc_score(y_test, y_proba)
    if auc_score > best_auc:
        best_auc = auc_score
        best_model = model
        best_model_name = name

print(f"Лучшая модель по ROC-AUC: {best_model_name} (AUC = {best_auc:.4f})")

#Рисуем графики

y_pred_best = best_model.predict(X_test)
y_proba_best = best_model.predict_proba(X_test)[:, 1]

#ROC-кривая
fpr, tpr, _ = roc_curve(y_test, y_proba_best)
plt.figure(figsize=(6, 5))
plt.plot(fpr, tpr, label=f'ROC curve (AUC = {best_auc:.3f})')
plt.plot([0, 1], [0, 1], 'k--', label='Random classifier')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title(f'ROC Curve — {best_model_name}')
plt.legend()
plt.grid(True)
plt.savefig('artifacts/figures/roc_curve.png', dpi=150, bbox_inches='tight')
plt.close()

#Confusion Matrix
plt.figure(figsize=(5, 4))
ConfusionMatrixDisplay.from_predictions(y_test, y_pred_best, cmap='Blues')
plt.title(f'Confusion Matrix — {best_model_name}')
plt.savefig('artifacts/figures/confusion_matrix.png', dpi=150, bbox_inches='tight')
plt.close()

#Precision-Recall Curve
precision, recall, _ = precision_recall_curve(y_test, y_proba_best)
plt.figure(figsize=(6, 5))
plt.plot(recall, precision, label='Precision-Recall curve')
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title(f'Precision-Recall Curve — {best_model_name}')
plt.grid(True)
plt.savefig('artifacts/figures/pr_curve.png', dpi=150, bbox_inches='tight')
plt.close()

Лучшая модель по ROC-AUC: RandomForestClassifier (AUC = 0.9201)


<Figure size 500x400 with 0 Axes>

In [8]:
from sklearn.inspection import permutation_importance

#используем test для оценки важности
perm_imp = permutation_importance(
    rf_cv,             #здесь использую сразу лучшую модель полученную в прошлом блоке
    X_test, 
    y_test, 
    n_repeats=10,       
    random_state=42, 
    scoring='f1'        
)
    
indices = np.argsort(perm_imp.importances_mean)[::-1][:10]
feature_names = X.columns[indices]
importances = perm_imp.importances_mean[indices]
std = perm_imp.importances_std[indices]

#Визуализировал топ 10 признаков 

plt.figure(figsize=(8, 6))
plt.barh(range(len(importances)), importances[::-1], xerr=std[::-1], color='skyblue')
plt.yticks(range(len(importances)), [f for f in feature_names][::-1])
plt.xlabel('Permutation Importance (Δ F1)')
plt.title(f'Top-10 Important Features — {best_model_name}')
plt.gca().invert_yaxis()
plt.tight_layout()
plt.savefig('artifacts/figures/permutation_importance.png', dpi=150, bbox_inches='tight')
plt.close()

Для лучшей модели (RandomForestClassifier) была рассчитана permutation importance на тестовой выборке. Наиболее значимыми оказался признак: `f_16`. Остальные признаки вносят существенно меньший вклад, а некоторые — практически не влияют на предсказание.

In [9]:
#Создание недостающих артефактов

models_for_search = {
    "DecisionTreeClassifier": dt_cv,
    "RandomForestClassifier": rf_cv,
    "HistGradientBoostingClassifier": hgb_cv
}

search_summaries = {}
for name, grid_search in models_for_search.items():
    search_summaries[name] = {
        "best_params": grid_search.best_params_,
        "cv_score": float(grid_search.best_score_)
    }
with open('artifacts/search_summaries.json', 'w') as f:
    json.dump(search_summaries, f, indent=4)

joblib.dump(best_model, 'artifacts/best_model.joblib')

best_model_meta = {
    "model_name": best_model_name,
    "best_params": models_for_search[best_model_name].best_params_,  # берём из GridSearchCV
    "cv_score": float(models_for_search[best_model_name].best_score_),
    "test_metrics": metrics_dict[best_model_name]
}

with open('artifacts/best_model_meta.json', 'w') as f:
    json.dump(best_model_meta, f, indent=4)